In [113]:
from ema_workbench import load_results
import pandas as pd
import statsmodels.api as sm
from ema_workbench.analysis import prim
from ema_workbench import ema_logging
import numpy as np
desired_width = 320
pd.set_option('display.width', desired_width)
pd.set_option('display.max_columns',50)

In [114]:
experiments, outcomes = load_results('./results/run_1.tar.gz')

In [115]:
df_outcomes = pd.DataFrame({key: np.concatenate(value) for key, value in outcomes.items()})
outcome = {f'{key} {i + 1}': value[:, i] for key, value in outcomes.items() for i in range(value.shape[1])}
df_outcomes = pd.DataFrame(outcome)
df_outcomes.to_excel('results/firstcheck.xlsx')

In [116]:
number_of_steps = sum(df_outcomes.columns.str.startswith('RfR Total Costs'))

In [117]:
for i in range(1,6):
    df_outcomes[f'A.{i}_Expected_Anual_Dagame'] = (df_outcomes[[col for col in df_outcomes.columns if col.startswith(f'A.{i}_Expected Annual Damage')]].sum(axis=1))*(200/number_of_steps)
    df_outcomes = df_outcomes.drop(df_outcomes.columns[df_outcomes.columns.str.startswith(f'A.{i}_Expected Annual Damage')], axis=1)

    df_outcomes[f'A.{i}_Dike_Investment_Costs'] = df_outcomes[[col for col in df_outcomes.columns if col.startswith(f'A.{i}_Dike Investment Costs')]].sum(axis=1)
    df_outcomes = df_outcomes.drop(df_outcomes.columns[df_outcomes.columns.str.startswith(f'A.{i}_Dike Investment Costs')], axis=1)

    df_outcomes[f'A.{i}_Expected_Number_of_Deaths'] = df_outcomes[[col for col in df_outcomes.columns if col.startswith(f'A.{i}_Expected Number of Deaths')]].sum(axis=1)*(200/number_of_steps)
    df_outcomes = df_outcomes.drop(df_outcomes.columns[df_outcomes.columns.str.startswith(f'A.{i}_Expected Number of Deaths')], axis=1)

df_outcomes['RfR_Total_Costs'] = df_outcomes[[col for col in df_outcomes.columns if col.startswith('RfR Total Costs')]].sum(axis=1)
df_outcomes = df_outcomes.drop(df_outcomes.columns[df_outcomes.columns.str.startswith('RfR Total Costs')], axis=1)

df_outcomes['Expected_evacuation_costs'] = df_outcomes[[col for col in df_outcomes.columns if col.startswith('Expected Evacuation Costs')]].sum(axis=1)
df_outcomes = df_outcomes.drop(df_outcomes.columns[df_outcomes.columns.str.startswith('Expected Evacuation Costs')], axis=1)

df_outcomes['Total_Expected_Number_of_Deaths'] = df_outcomes[[col for col in df_outcomes.columns if col.endswith('Expected_Number_of_Deaths')]].sum(axis=1)
df_outcomes['Total_Dike_Investment_Costs'] = df_outcomes[[col for col in df_outcomes.columns if col.endswith('Dike_Investment_Costs')]].sum(axis=1)
df_outcomes['Total_Expected_Anual_Dagame'] = df_outcomes[[col for col in df_outcomes.columns if col.endswith('Expected_Anual_Dagame')]].sum(axis=1)



In [118]:
df_outcomes.head()

,A.1_Expected_Anual_Dagame,A.1_Dike_Investment_Costs,A.1_Expected_Number_of_Deaths,A.2_Expected_Anual_Dagame,A.2_Dike_Investment_Costs,A.2_Expected_Number_of_Deaths,A.3_Expected_Anual_Dagame,A.3_Dike_Investment_Costs,A.3_Expected_Number_of_Deaths,A.4_Expected_Anual_Dagame,A.4_Dike_Investment_Costs,A.4_Expected_Number_of_Deaths,A.5_Expected_Anual_Dagame,A.5_Dike_Investment_Costs,A.5_Expected_Number_of_Deaths,RfR_Total_Costs,Expected_evacuation_costs,Total_Expected_Number_of_Deaths,Total_Dike_Investment_Costs,Total_Expected_Anual_Dagame
0,0.000000e+00,0,0.000000,5.368575e+10,0,52.367801,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0,52.367801,0,5.368575e+10
1,1.541560e+11,0,120.874582,2.760771e+09,0,2.858578,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0,123.733160,0,1.569167e+11
2,0.000000e+00,0,0.000000,1.264342e+10,0,12.291364,1.219810e+11,0,215.721831,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0,228.013194,0,1.346245e+11
3,1.498913e+11,0,121.284316,0.000000e+00,0,0.000000,0.000000e+00,0,0.000000,1.544127e+08,0,0.072141,0.0,0,0.0,0.0,0.0,121.356457,0,1.500457e+11
4,1.413886e+11,0,116.947475,1.005578e+09,0,1.147093,1.239573e+09,0,2.732778,8.597644e+08,0,0.493334,0.0,0,0.0,0.0,0.0,121.320680,0,1.444936e+11


In [109]:
df_experiments = experiments.drop(['policy', 'model', 'scenario'], axis = 1)
df_experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail', 'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate', 'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax', 'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1', 'discount rate 2', 'discount rate 3', '0_RfR 0',
       '0_RfR 1', '0_RfR 2', '0_RfR 3', '1_RfR 0', '1_RfR 1', '1_RfR 2', '1_RfR 3', '2_RfR 0', '2_RfR 1', '2_RfR 2', '2_RfR 3', '3_RfR 0', '3_RfR 1', '3_RfR 2', '3_RfR 3', '4_RfR 0', '4_RfR 1', '4_RfR 2', '4_RfR 3', 'EWS_DaysToThreat', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.1_DikeIncrease 3', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.2_DikeIncrease 3', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.3_DikeIncrease 3', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.4_DikeIncrease 3', 'A.5_DikeIncrease 0',
       'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2', 'A.5_DikeIncrease 3']

In [124]:
# #choosing only the columns of interest
# df_input_experiments = df_experiments[['0_RfR 1', '0_RfR 2', '0_RfR 3', '1_RfR 0', '1_RfR 1', '1_RfR 2', '1_RfR 3', '2_RfR 0', '2_RfR 1', '2_RfR 2', '2_RfR 3', '3_RfR 0', '3_RfR 1', '3_RfR 2', '3_RfR 3', '4_RfR 0', '4_RfR 1', '4_RfR 2', '4_RfR 3', 'EWS_DaysToThreat', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
#        'A.1_DikeIncrease 3', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.2_DikeIncrease 3', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.3_DikeIncrease 3', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.4_DikeIncrease 3', 'A.5_DikeIncrease 0',
#        'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2', 'A.5_DikeIncrease 3']].copy()

In [128]:
final_lhs = df_outcomes['Total_Expected_Number_of_Deaths']
X_0 = sm.add_constant(df_experiments)
est = sm.OLS(final_lhs, X_0.astype(float)).fit()
print(est.summary())
print(est.params)

                                   OLS Regression Results                                  
Dep. Variable:     Total_Expected_Number_of_Deaths   R-squared:                       1.000
Model:                                         OLS   Adj. R-squared:                    nan
Method:                              Least Squares   F-statistic:                       nan
Date:                             Wed, 07 Jun 2023   Prob (F-statistic):                nan
Time:                                     09:15:08   Log-Likelihood:                 278.52
No. Observations:                               10   AIC:                            -537.0
Df Residuals:                                    0   BIC:                            -534.0
Df Model:                                        9                                         
Covariance Type:                         nonrobust                                         
                              coef    std err          t      P>|t|      [0.025 

C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\lvanv\AppData\Local\Programs\Pyt

In [126]:
final_lhs = df_outcomes['Total_Dike_Investment_Costs']
X_0 = sm.add_constant(df_input_experiments)
est = sm.OLS(final_lhs, X_0.astype(float)).fit()
print(est.summary())
print(est.params)

                                 OLS Regression Results                                
Dep. Variable:     Total_Dike_Investment_Costs   R-squared:                         nan
Model:                                     OLS   Adj. R-squared:                    nan
Method:                          Least Squares   F-statistic:                       nan
Date:                         Wed, 07 Jun 2023   Prob (F-statistic):                nan
Time:                                 09:06:21   Log-Likelihood:                    inf
No. Observations:                           10   AIC:                              -inf
Df Residuals:                                9   BIC:                              -inf
Df Model:                                    0                                         
Covariance Type:                     nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:956: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels

In [127]:
final_lhs = df_outcomes['Total_Expected_Anual_Dagame']
X_0 = sm.add_constant(df_input_experiments)
est = sm.OLS(final_lhs, X_0.astype(float)).fit()
print(est.summary())
print(est.params)


                                 OLS Regression Results                                
Dep. Variable:     Total_Expected_Anual_Dagame   R-squared:                       0.000
Model:                                     OLS   Adj. R-squared:                  0.000
Method:                          Least Squares   F-statistic:                       nan
Date:                         Wed, 07 Jun 2023   Prob (F-statistic):                nan
Time:                                 09:07:47   Log-Likelihood:                -261.62
No. Observations:                           10   AIC:                             525.2
Df Residuals:                                9   BIC:                             525.5
Df Model:                                    0                                         
Covariance Type:                     nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\lvanv\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
